In [1]:
class Screen(object):

    # Construtor da classe
    def __init__(self, title, bgColor, width, height):
        self.title = title       # título da janela
        self.bgColor = bgColor   # cor de fundo
        self.width = width       # largura da janela
        self.height = height     # altura da janela
        self.screen = pygame.display.set_mode(self.size()) # define o tamanho da tela
        pygame.display.set_caption(self.title)             # define o título da janela
        
    # Executa o pipeline gráfico
    def run(self, obj):
        while True:  # laço principal
            # captura eventos
            for event in pygame.event.get(): 
                
                # Captura evento de clicar em botão para fechar
                if event.type == pygame.QUIT:
                    return pygame.quit()
            
            # preencha a tela com a cor de fundo
            self.screen.fill(self.bgColor)
            
            # gera o desenho
            obj.draw(self);
            
            # aplica o antialiasing
            #self.meanFilter()
            
            # atualiza a tela 
            pygame.display.update()
        
        
    # retorna um vetor com o tamanho da tela
    def size(self):
        return (self.width, self.height)
    
    # modifica um pixel na tela com a cor desejada
    def setPixel(self, x, y, color):
        self.screen.set_at((x, y), color)
    
    # filtro da média para o antialising
    def meanFilter(self):
        # Captura a matrix da tela
        #frameBuffer2 = pygame.PixelArray(self.screen)
        
        from copy import copy
        frameBuffer = pygame.surfarray.array3d(self.screen)
        #print(frameBuffer)
        
        import numpy as np
        mask = np.ones((3, 3)) * 1/9 
        
        #print(mask)
        temp = np.zeros((3))
               
        for i in range(1, self.width - 1):
            for j in range(1, self.height - 1):    
                subI = frameBuffer[i-1:i+2,j-1:j+2, 0]
                temp[0] = np.sum(np.multiply(subI, mask))
                #display(temp)
                
                subI = frameBuffer[i-1:i+2,j-1:j+2, 1]
                temp[1] = np.sum(np.multiply(subI, mask))
                #display(temp)
                
                subI = frameBuffer[i-1:i+2,j-1:j+2, 2]
                temp[2] = np.sum(np.multiply(subI, mask))
                #display(temp)
                
                #temp = np.zeros((3))
                
                #for k in range(-1,2):
                #    for l in range(-1,2):
                #        for b in range(3):
                #            temp[b] = temp[b] + frameBuffer[i + k][j + l][b] * mask[k + 1][l + 1]
                        
                #print(pygame.Color(int(temp[0]), int(temp[1]), int(temp[2]), 255))
                        
                self.setPixel(i, j, pygame.Color(int(temp[0]), int(temp[1]), int(temp[2]), 255));
                #frameBuffer2[i][j] = pygame.Color(int(temp[0]), int(temp[1]), int(temp[2]), 255);

In [2]:
class Line(object):
    # construtor da classe
    def __init__(self, x1, y1, x2, y2, color):
        self.x1 = x1       # coordenada x do primeiro ponto
        self.x2 = x2       # coordenada x do segundo ponto
        self.y1 = y1       # coordenada y do primeiro ponto
        self.y2 = y2       # coordenada y do segundo ponto
        self.color = color # cor do objeto
    
    # renderiza a linha desejada na tela
    def draw(self, screen):
        self.dda(screen)

    # Algoritmo DDA
    def dda(self, screen):      
        # Definição e Inicialização de Variáveis locais
        dx, dy, k = 0, 0, 0
        x_inc, y_inc = 0.0, 0.0
        x, y = 0.0, 0.0
    
        # Define os deslocamentos nas direções x e y
        dx = self.x2 - self.x1
        dy = self.y2 - self.y1
    
        # Define qual a direção de incremento fixo
        if abs(dx) > abs(dy):
            iter = abs(dx)
        else:
            iter = abs(dy)
        
        # Define os incrementos para cada direção
        x_inc = dx/iter
        y_inc = dy/iter

        # Define o ponto inicial
        x = self.x1
        y = self.y1

        # Desenha o ponto inicial na tela
        screen.setPixel(round(x), round(y), self.color)

        # Geração e renderização dos pontos seguintes da linha
        for k in range(iter):
            # Gera o próximo ponto
            x = x + x_inc
            y = y + y_inc
            
            # Desenha o ponto
            screen.setPixel(round(x), round(y), self.color)
            
    def bresenham(self, screen):
        # Definição e inicialização de variáveis locais
        dx, dy, d = 0, 0, 0
        incrE, incrNE = 0, 0
        x, y, xFinal = 0, 0, 0
        
        # Define os deslocamentos absolutos nas direções x e y
        dx = abs(self.x2 - self.x1)
        dy = abs(self.y2 - self.y1)
        
        # Define o d de teste inicial
        d = 2 * dy - dx
        
        # Define os incrementos nas direções x e y
        incrE = 2 * dy
        incrNE = 2 * (dy - dx)
        
        # Troca a ordem dos pontos em caso de segundo ponto à esquerda de primeiro ponto
        if self.x1 > self.x2:
            x = self.x2
            y = self.y2
            xFinal = self.x1
        else:
            x = self.x1
            y = self.y1
            xFinal = self.x2
        
        # Desenha o ponto inicial na tela
        screen.setPixel(x, y, self.color)
        
        # Gera e renderiza os pontos seguintes da linha
        while x < xFinal:
            # Gera o próximo ponto
            x = x + 1
            
            if d < 0:
                d = d + incrE
            else:
                y = y + 1
                d = d + incrNE
            
            # Desenha o próximo ponto
            screen.setPixel(x, y, self.color)

In [3]:
class EdgeInfo(object):
    # Construtor da Classe
    def __init__(self, initialPoint, finalPoint):
        if initialPoint.y <= finalPoint.y:
            self.yMax = finalPoint.y
            self.x = initialPoint.x  # x corrente, inicialmente x in Ymin
            self.yMin = initialPoint.y
        else:
            self.yMax = initialPoint.y
            self.x = finalPoint.x     # x corrente, inicialmente x in Ymin
            self.yMin = finalPoint.y
            
        self.inverseOfAngularCoefficient = (finalPoint.x - initialPoint.x) \
                                            / (finalPoint.y - initialPoint.y)
            
    @property
    def yMax(self):
        return self._yMax
    
    @yMax.setter
    def yMax(self, yMax):
        self._yMax = yMax
    
    @property
    def yMin(self):
        return self._yMin
    
    @yMin.setter
    def yMin(self, yMin):
        self._yMin = yMin
    
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, x):
        self._x = x
    
    def updateX(self):
        self.x = self.x + self.inverseOfAngularCoefficient

In [4]:
class Point2D(object):
    # construtor da Classe
    def __init__(self, x, y, color):
        self.x = x
        self.y = y
        self.color = color
    
    # Renderiza um ponto
    def draw(self, screen):
        screen.setPixel(self.x, self.y, self.color)
        
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, x):
        self._x = x
    
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, y):
        self._y = y

In [5]:
class Polygon(object):
    # construtor da classe
    def __init__(self, showEdges, edgeColor, isFilled, fillColor, estilo):
        self.listOfPoints = []
        self.showEdges = showEdges
        self.edgeColor = edgeColor
        self.isFilled = isFilled
        self.fillColor = fillColor
        self.estilo = estilo
    
    # Adiciona vértices na lista
    def addVertex(self, point):
        self.listOfPoints.append(point)
    
    # Renderiza Polígono
    def draw(self, screen):
        if len(self.listOfPoints) < 3:
            print("Não forma polígono. Menos de 3 vértices.")
            return
            
        # Desenha arestas se desejar
        if self.isFilled:
            self.scanline(screen)
        
        if self.showEdges:
            for i in range(0,len(self.listOfPoints) - 1):     
                pI = self.listOfPoints[i]
                pF = self.listOfPoints[i+1]
                line = Line(pI.x, pI.y, pF.x, pF.y, self.edgeColor)
                line.draw(screen)
            
            pI = self.listOfPoints[-1]
            pF = self.listOfPoints[0]
            line = Line(pI.x, pI.y, pF.x, pF.y, self.edgeColor)
            line.draw(screen)
            
            for i in range(0,len(self.listOfPoints)):               
                self.listOfPoints[i].draw(screen)
    
    # Faz o scanline para preencher o polígono
    def scanline(self, screen): 
        yMax = self.listOfPoints[0].y
        for item in self.listOfPoints:
            if item.y > yMax:
                yMax = item.y
                
        y = yMax # armazena o y corrente, começando pelo valor mínimo
        
        #### Cria tabela de arestas ####
        edgeTable = []
        for i in range (0, yMax+1):
            edgeTable.append([])
            
        for i in range(0,len(self.listOfPoints) - 1):     
        #for i in range(0,1):     
            # exclui arestas horizontais
            if self.listOfPoints[i].y - self.listOfPoints[i+1].y != 0:
                edge = EdgeInfo(self.listOfPoints[i], self.listOfPoints[i+1])
                yMin = edge.yMin
                if yMin < y:
                    y = yMin
    
                edgeTable[yMin].append(edge)       
        
        # Fecha o polígono 
        # exclui arestas horizontais
        if self.listOfPoints[-1].y - self.listOfPoints[0].y != 0:
            edge = EdgeInfo(self.listOfPoints[-1], self.listOfPoints[0])
            yMin = edge.yMin
            if yMin < y:
                y = yMin
            edgeTable[yMin].append(edge)
      
        ####
        activeET = []
        
        ### Laço principal
        while y <= yMax:
            #print(y)
                    
            # Move a lista y na ET para AET (ymin = y), mantendo a AET ordenada em x
            activeET.extend(edgeTable[y])
            edgeTable[y] = []
            activeET.sort(key = sortByX)
            
            # Desenhe os pixels do bloco na linha de varredura y, 
            # usando os pares de coordenadas x da AET (cada dois nós definem um bloco)
            for i in range(0, len(activeET) - 1, 2):
                for x in range(int(activeET[i].x), int(activeET[i + 1].x + 1)):

                    if self.estilo[x % len(self.estilo) - 1][y % len(self.estilo) - 1] == 1:
                        screen.setPixel(x, y, self.fillColor)
            
            # Atualiza o valor de y para a próxima linha de varredura
            y = y + 1
            
            # Remova as arestas que possuem ymax = y da AET
            delL = []
            for item in activeET:
                if item.yMax <= y:
                    delL.append(item)
            
            for item in delL:
                activeET.remove(item)
            
            delL.clear()
            
            # Para cada aresta na AET, atualize x = x + 1/m
            for item in activeET:
                item.updateX()
                
                
    def empty(self, ET):
        for item in ET:
            if item:
                return False
        
        return True

# Usada para ordenar a AET por valores de x
# Observe que a função não pertence à classe Polygon
def sortByX(item):
    return item.x
    

In [6]:
import pygame

pygame.init()   


pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

In [7]:
# screen = Screen("Tela", pygame.Color(255, 255, 255, 255), 700, 700)

# pol = Polygon(True, pygame.Color(255, 0, 0, 255), True, pygame.Color(255, 255, 0, 255))
# pol.addVertex(Point2D(20, 30, pygame.Color(0, 0, 0, 255)))
# pol.addVertex(Point2D(70, 10, pygame.Color(0, 0, 0, 255)))
# pol.addVertex(Point2D(130, 50, pygame.Color(0, 0, 0, 255)))
# pol.addVertex(Point2D(130, 100, pygame.Color(0, 0, 0, 255)))
# pol.addVertex(Point2D(70, 70, pygame.Color(0, 0, 0, 255)))
# pol.addVertex(Point2D(20, 90, pygame.Color(0, 0, 0, 255)))

# # Execução do Programa
# screen.run(pol)

In [8]:
class Picture(object):
    # Construtor da Classe
    def __init__(self):
        self.primitivas = [] # Define uma lista de primitivas para representar um desenho
    
    def draw(self, screen):
        
        # Insere primitivas na lista

        pol = Polygon(True, pygame.Color(255, 0, 0, 255), True, pygame.Color(255, 55, 0, 255), [[1,1,1,1,1,1],[1,1,1,1,1,1],[0,0,1,1,0,0],[0,0,1,1,0,0],[1,1,1,1,1],[1,1,1,1,1,1]])
        pol.addVertex(Point2D(65, 15, pygame.Color(0, 0, 0, 255)))
        pol.addVertex(Point2D(180, 50, pygame.Color(0, 0, 0, 255)))
        pol.addVertex(Point2D(200, 200, pygame.Color(0, 0, 0, 255)))

        pol2 =Polygon(True, pygame.Color(255, 0, 0, 255), True, pygame.Color(55, 155, 0, 255), [[1,1,1,1],[1,1,1,1],[0,0,0,0],[0,0,0,0]])
        pol2.addVertex(Point2D(20, 400, pygame.Color(0, 0, 0, 255)))
        pol2.addVertex(Point2D(260,500, pygame.Color(0, 0, 0, 255)))
        pol2.addVertex(Point2D(250, 600, pygame.Color(0, 0, 0, 255)))

        pol3 =Polygon(True, pygame.Color(255, 0, 0, 255), True, pygame.Color(55, 0, 255, 255), [[1,1,0,0,1],[1,1,0,0,1],[0,0,0,0,0],[0,0,0,0,0],[1,1,0,0,1]])
        pol3.addVertex(Point2D(350, 350, pygame.Color(0, 0, 0, 255)))
        pol3.addVertex(Point2D(350,500, pygame.Color(0, 0, 0, 255)))
        pol3.addVertex(Point2D(650, 400, pygame.Color(0, 0, 0, 255)))
        
        self.primitivas.append(pol);
        self.primitivas.append(pol2);
        self.primitivas.append(pol3);

        
        # Desenha cada primitiva que está na lista
        for item in self.primitivas:
            item.draw(screen);
        

In [9]:
# Criação do Objeto de Tela
screen = Screen("Tela", pygame.Color(255, 255, 255, 255), 700, 700)

# executa o desenho
pic = Picture();

screen.run(pic);